In [1]:
import json
import csv
import itertools
import pandas as pd

In [2]:
# import ijson

# events_data = []
filename = "/Users/chitrankdixit/Downloads/data.json"
# with open(filename, 'r') as f:
#     objects = ijson.items(f, '')

#     for row in objects:
#         events_data.append(row)

In [3]:
df = pd.read_json(filename)

In [4]:
df.head()

,browser,browser_version,city,day_advance,device,distinct_id,duration_of_booking,event,initial_referrer,parent_id,price,referrer,time,utm_campaign,utm_medium,utm_source
0,Firefox,54.0,Mumbai,96,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES05834,1999,http://flyrobe.com/women/lehengas?delivery_dat...,1503533058,NaN,NaN,NaN
1,Firefox,54.0,Mumbai,96,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES01768,2699,http://flyrobe.com/?gclid=Cj0KEQjwz_TMBRD0jY-R...,1503533208,NaN,NaN,NaN
2,Firefox,54.0,Mumbai,107,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES01768,2699,http://flyrobe.com/?gclid=Cj0KEQjwz_TMBRD0jY-R...,1503533219,NaN,NaN,NaN
3,Mobile Safari,10.0,Mumbai,10,iPhone,15c82f54f9017e-0dc211ce35bbbc-64793c7f-3d10d-1...,4.0,Add to Cart Web,https://www.google.co.in/,FRES05560,3899,https://flyrobe.com/women/ethnic-gowns,1503533525,NaN,NaN,NaN
4,Chrome,59.0,Delhi,6,Android,15cb6e4205211-08b6c197de6bc7-3f112b20-38400-15...,4.0,Add to Cart Web,https://www.google.co.in/,FRES03765,1499,https://www.google.co.in/,1503536564,NaN,NaN,NaN


In [6]:
# distinct id of the users who has added item to the cart
user_distinct_ids = df[df.event == "Add to Cart Web"]

In [7]:
user_distinct_ids.head()

,browser,browser_version,city,day_advance,device,distinct_id,duration_of_booking,event,initial_referrer,parent_id,price,referrer,time,utm_campaign,utm_medium,utm_source
0,Firefox,54.0,Mumbai,96,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES05834,1999,http://flyrobe.com/women/lehengas?delivery_dat...,1503533058,NaN,NaN,NaN
1,Firefox,54.0,Mumbai,96,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES01768,2699,http://flyrobe.com/?gclid=Cj0KEQjwz_TMBRD0jY-R...,1503533208,NaN,NaN,NaN
2,Firefox,54.0,Mumbai,107,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES01768,2699,http://flyrobe.com/?gclid=Cj0KEQjwz_TMBRD0jY-R...,1503533219,NaN,NaN,NaN
3,Mobile Safari,10.0,Mumbai,10,iPhone,15c82f54f9017e-0dc211ce35bbbc-64793c7f-3d10d-1...,4.0,Add to Cart Web,https://www.google.co.in/,FRES05560,3899,https://flyrobe.com/women/ethnic-gowns,1503533525,NaN,NaN,NaN
4,Chrome,59.0,Delhi,6,Android,15cb6e4205211-08b6c197de6bc7-3f112b20-38400-15...,4.0,Add to Cart Web,https://www.google.co.in/,FRES03765,1499,https://www.google.co.in/,1503536564,NaN,NaN,NaN


In [8]:
user_distinct_ids = user_distinct_ids['distinct_id'].tolist()

In [9]:
user_distinct_ids[0]

'1573d7c483eb3-06235568306ea58-4c594331-100200-1573d7c484011c'

In [10]:
user_events = df[df.distinct_id.isin(user_distinct_ids)]

In [11]:
user_events = user_events.sort_values(by=['time'])

In [14]:
user_events.head()

,browser,browser_version,city,day_advance,device,distinct_id,duration_of_booking,event,initial_referrer,parent_id,price,referrer,time,utm_campaign,utm_medium,utm_source
1591546,Chrome,56.0,Delhi,NaN,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,NaN,Details Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512008,NaN,NaN,NaN
80557,Chrome,56.0,Delhi,22,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,8.0,Add to Cart Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512045,NaN,NaN,NaN
420251,Mobile Safari,10.0,Mumbai,NaN,iPad,blissglobalkidzee@gmail.com,NaN,Details Web,$direct,FRES01492,149,NaN,1486512054,NaN,NaN,NaN
1239440,Chrome,56.0,Delhi,NaN,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,NaN,Details Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512056,NaN,NaN,NaN
769396,Chrome,56.0,Delhi,NaN,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,NaN,Details Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512058,NaN,NaN,NaN


In [15]:
list1 = []; user_dict = {}
for index, each in user_events.iterrows():
    if each["distinct_id"] not in user_dict:
        user_dict[each["distinct_id"]] = [{each["event"]: [each["time"], each["parent_id"]]}]
    else:
        user_dict[each["distinct_id"]].append({each["event"]: [each["time"], each["parent_id"]]})

In [16]:
user_dict

{'15a19d486641d-05e311073b4267-57e1b3c-100200-15a19d486651cb': [{'Details Web': [1486512008,
    'FRES02262']},
  {'Add to Cart Web': [1486512045, 'FRES02262']},
  {'Details Web': [1486512056, 'FRES02262']},
  {'Details Web': [1486512058, 'FRES02262']},
  {'Add to Cart Web': [1486512077, 'FRES02262']}],
 'blissglobalkidzee@gmail.com': [{'Details Web': [1486512054, 'FRES01492']},
  {'Details Web': [1486512202, 'FRES01998']},
  {'Details Web': [1486512223, 'FRES01494']},
  {'Details Web': [1486512288, 'FRES01498']},
  {'Details Web': [1486512338, 'FRES01548']},
  {'Details Web': [1486512635, 'FRES03785']},
  {'Details Web': [1486512804, 'FRES01733']},
  {'Details Web': [1486513171, 'FRES01498']},
  {'Details Web': [1486513189, 'FRES03785']},
  {'Details Web': [1486933978, 'FRES01733']},
  {'Details Web': [1486934157, 'FRES04214']},
  {'Details Web': [1486934241, 'FRES04214']},
  {'Details Web': [1486934250, 'FRES01737']},
  {'Details Web': [1486934382, 'FRES03785']},
  {'Details Web': [1

In [13]:
record_inv_detail = {}
record_inv_atc = {}
for user in user_dict:
    # TODO: arrange the events
    record_inv_detail[user] = []
    record_inv_atc[user] = []
    for event in user_dict[user]:
        for key, value in event.items():
            if key == "Details Web":
                record_inv_detail[user].append(value)
            if key == "Add to Cart Web":
                record_inv_detail[user].append(value)
                if (record_inv_detail[user][-1][0] - record_inv_detail[user][0][0] < 2000):
                    record_inv_atc[user].append([record_inv_detail[user][0], record_inv_detail[user][-1]])
                    record_inv_detail[user].clear()
                else:
                    record_inv_detail[user].clear()

In [14]:
fdp_atc = []
for user, list_item in record_inv_atc.items():
    for cycle in list_item:
        list_1 = []
        for entry in cycle:
            list_1 += [entry[1]]
        fdp_atc.append(list_1)

In [ ]:
look_up = {}
with open('/Users/chitrankdixit/Downloads/look_up.csv', 'r') as f:
    reader = csv.reader(f)
    for counter, row in enumerate(reader):
        if counter !=0:
            #import pdb;pdb.set_trace()
            if row[1] not in look_up.keys():
                look_up[row[1]] = [{"inventory_category": row[6], "brand_id": row[4]}]
            else:
                look_up[row[1]].append({"inventory_category": row[6], "brand_id": row[4]})
            
# later update this lookup
# fetch rentals from rds 

> <ipython-input-15-c40e675d7fcc>(7)<module>()
-> if row[1] not in look_up.keys():
(Pdb) row[1]
'FRES05299'
(Pdb) row[4]
'156'
(Pdb) c
> <ipython-input-15-c40e675d7fcc>(6)<module>()
-> import pdb;pdb.set_trace()
(Pdb) look_up
{'FRES05299': [{'inventory_category': 'Kurta sets', 'brand_id': '156'}]}
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
(Pdb) c
> <ipython-input-15-c40e675d7fcc>(7)<module>()
-> if row[1] not in look_up.keys():
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--


In [19]:
inv_cat_list = []
for element in fdp_atc:
    list_2 = []
    for each in element:
        #get inv_cat for the item : inv_cat_item
        inv_cat_item=""
        for item in look_up:
            if item[1] == each:
                inv_cat_item = item[6]
        list_2 += [inv_cat_item]
    inv_cat_list.append(list_2)

In [20]:
list_of_cat   A    B     C

A   1
B       1
C           1
inv_cat_dict = {}
for each in inv_cat_list:
    if each[0] not in inv_cat_dict:
        inv_cat_dict[each[0]] = [each[1]]
    else:
        inv_cat_dict[each[0]] += [each[1]]

In [21]:
inv_cat_dict

{'': ['', '', '', '', '', '', '', '', '', '', '', '', 'Indo Western', '', ''],
 'Anarkali': ['Kurta sets',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Lehenga',
  'Lehenga',
  'Anarkali',
  'Gown',
  'Lehenga',
  'Anarkali',
  'Kurta Pyjama',
  'Anarkali',
  'Anarkali',
  'Jumpsuit and Playsuit',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Gown',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Kurta sets',
  'Lehenga',
  'Anarkali',
  'Anarkali',
  'Lehenga',
  'Anarkali',
  'Anarkali',
  'Bags',
  'Anarkali',
  'Anarkali',
  'Maxi Dress',
  'Jodhpuri suits',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Anarkali',
  'Sarees',
  '

In [ ]:
with open('/home/flyrobe/Documents/elasticsearch_to_json/bronze/inventory_category.json', 'w') as outfile:
    json.dump(inv_cat_dict, outfile)

In [17]:
#user_grouped_events = user_events.groupby( [ "distinct_id", "event", "time", "parent_id"] )

In [40]:
#user_grouped_events = user_grouped_events[['event','distinct_id', 'time', 'parent_id']]

In [1]:
#user_grouped_events.size()

NameError: name 'user_grouped_events' is not defined